## Recommendation System
Collaborative filtering with implicit feedback based on latent factors. Prepare data on user-item relationships for each user-company in format that ALS can use.
We require each unique assignee ID in the rows of the matrix, and each unique item ID in columns of matrix.
Values of matrix should be (?) binary user-item preference * confidence

In [1]:
import pyspark
import pyspark.sql.functions as F
from pyspark.sql import Row
from pyspark.sql.types import ArrayType, IntegerType
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

from lightfm import LightFM
from lightfm.datasets import fetch_movielens
from lightfm.evaluation import precision_at_k

import pandas as pd
import numpy as np

from test_model import (get_patent_fields_list, get_ml_patents, 
                        create_title_abstract_col,trim_data, 
                        structure_dataframe, partition_dataframe, 
                        build_pipeline, process_docs, pat_inv_map, get_topics)

from rec_system import alphanum_to_int, int_to_alphanum

import gensim
import gensim.corpora as corpora
from gensim.corpora import Dictionary, mmcorpus
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.models.phrases import Phrases, Phraser
from gensim.models.ldamodel import LdaModel
from gensim.models import AuthorTopicModel
from gensim.test.utils import common_dictionary, datapath, temporary_file
from smart_open import smart_open

import spacy
from spacy.lemmatizer import Lemmatizer
from spacy.lang.en import LEMMA_INDEX, LEMMA_EXC, LEMMA_RULES

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, punkt, RegexpTokenizer, wordpunct_tokenize
from nltk.stem import PorterStemmer, LancasterStemmer, WordNetLemmatizer

import json
from pandas.io.json import json_normalize
import requests
import re
import os
import calendar
import requests
from bs4 import BeautifulSoup
import pickle
import math

import matplotlib.pyplot as plt
import pyLDAvis
import pyLDAvis.gensim

from pprint import pprint

%load_ext autoreload
%autoreload 2

/anaconda3/lib/python3.6/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn('LightFM was compiled without OpenMP support. '


In [2]:
spark = pyspark.sql.SparkSession.builder.getOrCreate()
spark

In [3]:
sc = spark.sparkContext
sc

<SparkContext master=local[*] appName=pyspark-shell>

### Data understanding - Acquire data

In [200]:
# load pickled dataset
with open('/Users/lee/Documents/techniche/techniche/data/raw_data_1000', 'rb') as f:
    raw_data_1000 = pickle.load(f)

In [201]:
# define desired keys/columns as criteria to subset dataset
retained_keys = ['patent_number', 'patent_firstnamed_assignee_id']

In [202]:
# subset raw dataset by desired keys/columns
data_1000 = trim_data(data=raw_data_1000, keys=retained_keys)

In [203]:
# create Pandas dataframe
df_1000 = pd.DataFrame(data_1000)

### Data preparation
Prepare data on user-item relationships for each user-company in format that ALS can use.
We require each unique assignee ID in the rows of the matrix, and each unique item ID in columns of matrix.
Values of matrix should be (?) binary user-item preference * confidence

In [204]:
# create new rating column and assign value of 1
df_1000['rating'] = 1

In [205]:
# drop row that contains invalid data in patent_number column
df_1000[df_1000.patent_number.str.contains('[RE]')]
df_1000 = df_1000.drop(df_1000.index[[717]])

In [206]:
# drop NaNs in patent_firstnamed_assignee_id column
df_1000.info()
df_1000 = df_1000.dropna()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 999 entries, 0 to 999
Data columns (total 3 columns):
patent_firstnamed_assignee_id    972 non-null object
patent_number                    999 non-null object
rating                           999 non-null int64
dtypes: int64(1), object(2)
memory usage: 31.2+ KB


In [207]:
# convert patent_number column from string to int
df_1000 = df_1000.astype({'patent_number': 'int64'})
# uncomment to confirm
# df_1000.info()

In [208]:
# convert alphanumeric patent_firstnamed_assignee_id col to int
df_1000 = df_1000.astype({'patent_number': 'int64'})
# s = 'org_VU2IXnxgxGIK8A8oQrwm'

# code = [ord(c) for c in s]
# code

In [209]:
df_1000.head(3)

,patent_firstnamed_assignee_id,patent_number,rating
0,org_VU2IXnxgxGIK8A8oQrwm,10226194,1
1,org_9cmRc2rH8nbl8O9VuxYL,10228278,1
2,org_8O8xQifxyiW5pZB2KuDx,10228693,1


In [210]:
hash('org_VU2IXnxgxGIK8A8oQrwm')

8569133740707573506

In [211]:
# df_1000['patent_firstnamed_assignee_id'] = df_1000['patent_firstnamed_assignee_id'].apply(hash).apply(abs)
df_1000['patent_firstnamed_assignee_id'] = df_1000['patent_firstnamed_assignee_id'].apply(hash).apply(abs)

In [212]:
df_1000 = df_1000.astype({'patent_firstnamed_assignee_id': 'int'})

In [213]:
df_1000.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 972 entries, 0 to 999
Data columns (total 3 columns):
patent_firstnamed_assignee_id    972 non-null int64
patent_number                    972 non-null int64
rating                           972 non-null int64
dtypes: int64(3)
memory usage: 30.4 KB


#### Data preparation - create Spark dataframe from pandas dataframe

In [214]:
sp_df_1000 = spark.createDataFrame(df_1000)

In [215]:
sp_df_1000.show()

+-----------------------------+-------------+------+
|patent_firstnamed_assignee_id|patent_number|rating|
+-----------------------------+-------------+------+
|          8569133740707573506|     10226194|     1|
|          3101878516446872278|     10228278|     1|
|          6145279804171940698|     10228693|     1|
|          2274901963694307659|     10228922|     1|
|          3560972456146974693|     10228931|     1|
|          4506043873413760726|     10229106|     1|
|          5637413039304462672|     10229109|     1|
|          2124294643000947350|     10229113|     1|
|          2435005256187647813|     10229148|     1|
|          2274901963694307659|     10229156|     1|
|          5637413039304462672|     10229173|     1|
|          2274901963694307659|     10229187|     1|
|          2274901963694307659|     10229189|     1|
|          5401471083817472319|     10229357|     1|
|          2274901963694307659|     10229368|     1|
|          6249349723877024109|     10229673| 

In [216]:
sp_df_1000.dtypes

[('patent_firstnamed_assignee_id', 'bigint'),
 ('patent_number', 'bigint'),
 ('rating', 'bigint')]

In [217]:
# TODO (Lee) - convert to int?

In [177]:
# split into 
(training, test) = sp_df_1000.randomSplit([0.8, 0.2])

In [178]:
training.show()

+-----------------------------+-------------+------+
|patent_firstnamed_assignee_id|patent_number|rating|
+-----------------------------+-------------+------+
|         -9221669585220209733|     10205728|     1|
|         -9086329284116169685|     10176575|     1|
|         -8989251368548248094|     10182097|     1|
|         -8989251368548248094|     10191889|     1|
|         -8964916847225960180|     10223586|     1|
|         -8852994261006315466|     10204627|     1|
|         -8658142159812273104|     10162315|     1|
|         -7708214383845801069|     10210861|     1|
|         -7522086197512374256|     10216505|     1|
|         -7427531594348498705|     10198791|     1|
|         -7427531594348498705|     10198846|     1|
|         -7380718460450320198|     10163003|     1|
|         -7280787859308227436|     10194866|     1|
|         -7064625980329811547|     10170104|     1|
|         -6916213623629676396|     10223499|     1|
|         -6845758941828871168|     10222315| 

### Model # 1

In [95]:
# Build the recommendation model using ALS on the training data
# Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
# set implicitPrefs to True to get better results b/c latent matrix 
# rank? alpha=0.01?
# build ALS model
als = ALS(maxIter=5,
          regParam=0.01, 
          implicitPrefs=True,
          userCol="patent_firstnamed_assignee_id", 
          itemCol="patent_number", 
          ratingCol="rating",
          coldStartStrategy="drop")

In [96]:
# fit the ALS model to the training set
model = als.fit(training)

Py4JJavaError: An error occurred while calling o347.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 11.0 failed 1 times, most recent failure: Lost task 0.0 in stage 11.0 (TID 11, localhost, executor driver): org.apache.spark.SparkException: Failed to execute user defined function($anonfun$4: (bigint) => int)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anonfun$13$$anon$1.hasNext(WholeStageCodegenExec.scala:636)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:409)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:409)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:409)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:390)
	at scala.collection.Iterator$class.foreach(Ite

#### Model #1 - Evaluation - Compare to naive baseline
Compare model evaluation result with naive baseline model that only outputs (for explicit - the average rating (or you may try one that outputs the average rating per movie).

#### Model #1 - Optimize model

In [ ]:
params = ParamGridBuilder().addGrid(als_model.regParam, [0.01,0.001,0.1]).addGrid(als_model.rank, [4,10,50]).build()


## instantiate crossvalidator estimator
cv = CrossValidator(estimator=als_model, estimatorParamMaps=params,evaluator=evaluator,parallelism=4)
best_model = cv.fit(movie_ratings)    

In [ ]:
# Getting Predictions for a New User

In [ ]:
predictions = model.transform(test)